# Введение в protobuf. Diving in protobuf   

## Определение
Protobuf (Protocol Buffers) — это механизм сериализации данных, разработанный компанией Google.

Использует HTTP/2, обеспечивая многопоточность и эффективное управление соединениями.

Позволяет компактно и эффективно сериализовать структура данных для обмена между различными системами/сервисами.


## Преимущества

### 1. Компактность

Использует бинарных формат для сериализации данных, что позволяет значительно уменьшить объем передиаваемых данных по сравнению с текстовыми форматами (XML, JSON).

### 2. Скорость

Быстрее в сериализации и десериализации данных по сравнению с текстовыми форматами.

### 3. Межъязыковая поддержка

Поддерживает множество языков программирования (Python, C++, Java, Go, Kotlin и другие), что позволяет интегрировать системы, написанные на разных языках.

## Общая схема работы gRPC

**Клиент и сервер**:

- Клиент вызывает удалённые процедуры с помощью RPC.

- Сервер обрабатывает запросы и возвращает результаты.

**Потоки данных**:

- Однаправленные или двусторонние.

- Поддержка streaming RPC.

**Преимущества HTTP/2**:

- Многопоточность

- Сжатие заголовков

- Поддержка стримов

## Ключевые черты Protocol Buffers

- Сериализация данных:

  - Сжатие и быстодействие.

  - Описание данных в .proto файлах.

- Компиляция:

  - Генерация кода (классы для конкретных языков).

  - Структура .proto файла:

```
syntax = "proto3";

message Example {
    int32 id = 1;
    string name = 2;
}
```

## Преимущества gRPC перед REST

- Высокая производительность

  - Уменьшение объёма данных.

- Стриминг

  - Поддержка двустороннего обмена данными (bidirectional streaming).

- Языковая и платформенная независимость.

- Четкое описание API: контракты задаются в .proto файлах, минимизируя вероятность ошибок в реализации клиента и сервера.



### Недостатки gRPC:

- Сложность отладки:

Данные передаются в бинарном формате, что делает их менее читаемыми для человека.

- Ограниченная поддержка браузерами:

Требуется специальный прокси для работы с браузерами, так как HTTP/2 не поддерживается напрямую всеми браузерами.

- Крутая кривая обучения:

Необходимо освоить Protocol Buffers и структуру .proto файлов.

- Ориентирован на внутренние системы:

Для публичных API REST чаще оказывается предпочтительным благодаря использованию привычных форматов JSON и HTTP/1.1.

## Практика. Hello, world

Проект ___hello_world_example_protobuf___ PyCharm

1. Создадим папку для проекта (с помощью ide или в консоли)

```shell
mkdir hello_world_example_protobuf
```

2. Создадим и активируем виртуальное окружение (с помощью virtualenv)

```shell
virtualenv env
source env/bin/activate
```

3. Создадим и активируем виртуальное окружение (с помощью virtualenv)

```shell
virtualenv env
. env/bin/activate
```

Обновим pip и установим зависимости:

```shell
python -m pip install --upgrade pip
python -m pip install grpcio
python -m pip install grpcio-tools
```

Создадим файлы внутри каталога с проектом файлы со следующим содержимым:
создадим каталог ```protobufs```
и в нем
```helloworld.proto```:

```proto
syntax = "proto3";

// The greeting service definition.
service Greeter {
  // Sends a greeting
  rpc SayHello (HelloRequest) returns (HelloReply) {}

  rpc SayHelloAgain (HelloRequest) returns (HelloReply) {}
}

// The request message containing the user's name.
message HelloRequest {
  string name = 1;
}

// The response message containing the greetings
message HelloReply {
  string message = 1;
}
```

```greeter_client.py```


```python
"""The Python implementation of the GRPC helloworld.Greeter client."""
from __future__ import print_function

import logging

import grpc
import helloworld_pb2
import helloworld_pb2_grpc


def run():
    # NOTE(gRPC Python Team): .close() is possible on a channel and should be
    # used in circumstances in which the with statement does not fit the needs
    # of the code.
    print("Will try to greet world ...")
    with grpc.insecure_channel('localhost:50051') as channel:
        stub = helloworld_pb2_grpc.GreeterStub(channel)
        response = stub.SayHello(helloworld_pb2.HelloRequest(name='you'))
        print("Greeter client received: " + response.message)
        response = stub.SayHelloAgain(helloworld_pb2.HelloRequest(name='you'))
        print("Greeter client received: " + response.message)


if __name__ == "__main__":
    logging.basicConfig()
    run()
```

```greeter_server.py```

```python
"""The Python implementation of the GRPC helloworld.Greeter server."""

from concurrent import futures
import logging

import grpc
import helloworld_pb2
import helloworld_pb2_grpc


class Greeter(helloworld_pb2_grpc.GreeterServicer):
    def SayHello(self, request, context):
        return helloworld_pb2.HelloReply(message="Hello, %s!" % request.name)

    def SayHelloAgain(self, request, context):
        return helloworld_pb2.HelloReply(message=f"Hello again, {request.name}!")


def serve():
    port = "50051"
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    helloworld_pb2_grpc.add_GreeterServicer_to_server(Greeter(), server)
    server.add_insecure_port("[::]:" + port)
    server.start()
    print("Server started, listening on " + port)
    server.wait_for_termination()


if __name__ == "__main__":
    logging.basicConfig()
    serve()
```

Необходимо сгененирировать на основе .proto-файла необходимые для передачи данных файлы (классы на Python для выполнения запросов и получения ответов).

```shell
python -m grpc_tools.protoc -I./protobufs --python_out=. --pyi_out=. --grpc_python_out=. ./protobufs/helloworld.proto
```


должны быть сгенерированы три файла:
- helloworld_pb2.py
- helloworld_pb2.pyi
- helloworld_pb2_grpc.py

Запустим в отдельной консоли скрипты на сервере и клиенте:

```python
python greeter_server.py
```

```python
python greeter_client.py
```

Добавим серверу функционала:

добавим код в файл ```helloworld.proto``` внутрь ```service Greeter```, чтобы он умел что-то ещё:

```proto
service Greeter {
...
rpc SayMyName (MyNameRequest) returns (MyNameReply) {}
...

...
message MyNameRequest {
  string name = 1;
}
message MyNameReply {
  string message = 1;
}

...

```

Перегенерируем _pb2 файлы:

```shell
python -m grpc_tools.protoc -I./protobufs --python_out=. --pyi_out=. --grpc_python_out=. ./protobufs/helloworld.proto
```

Добавим функционал в greeter_server.py:
```python
class Greeter(helloworld_pb2_grpc.GreeterServicer):
  ...
  def SayMyName(self, request, context):
        return helloworld_pb2.MyNameReply(message=f"— You're {request.name}.\n— You're goddamn right.")
```
и в ```greeter_client.py```:


```python
...
response = stub.SayMyName(helloworld_pb2.MyNameReply(message='Heisenberg'))
print(f"{response.message}")
...
```

Типы данных в proto-файлах:

В .proto файлах используется строго определённый набор типов данных, разделённый на несколько категорий: числовые, строковые, булевы, байтовые и комплексные типы.


```proto
syntax = "proto3";

package example;

// Перечисление (enum)
enum Status {
    UNKNOWN = 0;
    ACTIVE = 1;
    INACTIVE = 2;
}

// Основное сообщение
message Example {
    // Числовые типы
    int32 id = 1;            // Целое число
    float value = 2;         // Число с плавающей точкой
    sfixed64 long_value = 3; // Знаковое фиксированное число

    // Строки и байты
    string name = 4;         // Строка UTF-8
    bytes data = 5;          // Бинарные данные

    // Булевы значения
    bool is_valid = 6;       // Логическое значение

    // Перечисления
    Status current_status = 7; // Перечисление

    // Вложенные сообщения
    message Address {
        string city = 1;
        string street = 2;
        int32 house_number = 3;
    }

    Address address = 8; // Поле типа вложенного сообщения

    // Списки
    repeated int32 numbers = 9;     // Массив целых чисел
    repeated Address contacts = 10; // Массив вложенных объектов
}

```

Наиболее интересны комплексные типы:

- **Списки**: Использование repeated для создания массива.
- **Сообщения**: Пользовательские типы (наборы полей).
- **Вложенные сообщения и enum**: Для описания сложных структур данных.

__package example;__

Строка package в начале .proto файла не является обязательной, но её рекомендуется использовать в большинстве случаев, особенно в проектах с большим количеством сообщений и файлов.

Зачем нужна строка package?

1. Избежание конфликтов имен:

При использовании package, все сгенерированные классы и сообщения будут организованы в пространство имен (namespace) или модуль, связанный с этим пакетом. Это помогает избежать коллизий, когда два .proto файла определяют сообщения с одинаковыми именами.

2. Организация кода:
В больших проектах структура пакетов помогает логически группировать API.

3. Совместимость с различными языками:
В большинстве языков программирования пакеты преобразуются в пространства имен:
Для Python — это модули.
Например, для фрагмента

```proto
syntax = "proto3";

package example.myapi;

message Person {
    string name = 1;
    int32 age = 2;
}
```

это будет пакет:

```python
from example.myapi.person_pb2 import Person
```

Если вы хотите использовать в .proto файле тип данных, определённый в другом .proto файле, это делается с помощью директивы import. Такая ситуация часто возникает при работе с большими проектами, где типы данных описаны в отдельных файлах для модульности и повторного использования.

```proto
// common.proto
syntax = "proto3";

package common;

// Определение общего типа данных
message Address {
    string city = 1;
    string street = 2;
    int32 house_number = 3;
}

```

другой файл user.proto:

```proto
// user.proto
syntax = "proto3";

package user;

// Импортируем другой .proto файл
import "common.proto";

// Используем тип данных из другого файла
message User {
    string id = 1;          // Уникальный идентификатор пользователя
    string name = 2;        // Имя пользователя
    common.Address address = 3; // Адрес, использующий нестандартный тип данных
}
```

## Практика. Recommender Service

Комментарии по коду:


```shell
python -m grpc_tools.protoc -I ../protobufs --python_out=. \
--grpc_python_out=. ../protobufs/recommendations.proto
```

Запуск приложения:
```FLASK_APP=marketplace.py FLASK_ENV=development flask run```

```dockerfile
## recommendation service
FROM python

RUN mkdir /service
COPY protobufs/ /service/protobufs/
COPY recommendations/ /service/recommendations/
WORKDIR /service/recommendations
RUN python -m pip install --upgrade pip
RUN python -m pip install -r requirements.txt
RUN python -m grpc_tools.protoc -I ../protobufs --python_out=. \
           --grpc_python_out=. ../protobufs/recommendations.proto

EXPOSE 50051
ENTRYPOINT [ "python", "recommendations.py" ]
```

`docker build . -f recommendations/Dockerfile -t recommendations`




`docker run -p 127.0.0.1:50051:50051/tcp recommendations`

```dockerfile
## marketplace service
FROM python

RUN mkdir /service
COPY protobufs/ /service/protobufs/
COPY marketplace/ /service/marketplace/
WORKDIR /service/marketplace
RUN python -m pip install --upgrade pip
RUN python -m pip install -r requirements.txt
RUN python -m grpc_tools.protoc -I ../protobufs --python_out=. \
           --grpc_python_out=. ../protobufs/recommendations.proto

EXPOSE 5000
ENV FLASK_APP=marketplace.py
ENTRYPOINT [ "flask", "run", "--host=0.0.0.0"]
```

`docker build . -f marketplace/Dockerfile -t marketplace`

`docker run -p 127.0.0.1:5000:5000/tcp marketplace`

`docker network create microservices`





```
docker run -p 127.0.0.1:50051:50051/tcp --network microservices \
             --name recommendations recommendations
```




`recommendations_channel = grpc.insecure_channel("localhost:50051")`



```
recommendations_host = os.getenv("RECOMMENDATIONS_HOST", "localhost")
recommendations_channel = grpc.insecure_channel(
    f"{recommendations_host}:50051"
)
```



```shell
docker build . -f marketplace/Dockerfile -t marketplace
docker run -p 127.0.0.1:5000:5000/tcp --network microservices \
             -e RECOMMENDATIONS_HOST=recommendations marketplace
```



```yaml
version: "3.8"
services:

    marketplace:
        build:
            context: .
            dockerfile: marketplace/Dockerfile
        environment:
            RECOMMENDATIONS_HOST: recommendations
        image: marketplace
        networks:
            - microservices
        ports:
            - 5000:5000

    recommendations:
        build:
            context: .
            dockerfile: recommendations/Dockerfile
        image: recommendations
        networks:
            - microservices

networks:
    microservices:
    
```

```docker-compose up```

```python

# recommendations/recommendations_test.py
from recommendations import RecommendationService

from recommendations_pb2 import BookCategory, RecommendationRequest

def test_recommendations():
    service = RecommendationService()
    request = RecommendationRequest(
        user_id=1, category=BookCategory.MYSTERY, max_results=1
    )
    response = service.Recommend(request, None)
    assert len(response.recommendations) == 1
```

```python
from urllib.request import urlopen

def test_render_homepage():
    homepage_html = urlopen("http://localhost:5000").read().decode("utf-8")
    assert "<title>Online Books For You</title>" in homepage_html
    assert homepage_html.count("<li>") == 3

```

```sh
docker-compose build
docker-compose up
docker-compose exec marketplace pytest marketplace_integration_test.py
```